In [1]:
#импорт необходимых библиотек
import pandas as pd
import requests
import csv

In [2]:
from bs4 import BeautifulSoup

In [3]:
import random

In [4]:
import time

In [5]:
#pip install selenium

In [6]:
#pip install webdriver-manager

In [7]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [8]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\rus_k\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache
<ipython-input-8-c026f0c15647>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [9]:
# импорт библиотеки работы с базой данных
import sqlite3

In [10]:
# подключение к базе данных
connection = sqlite3.connect('apartments.db')
# создание объекта подключения
cursor = connection.cursor()
print("База данных успешно подключена к SQLite")

База данных успешно подключена к SQLite


In [11]:
# вывод наименований созданных таблиц
cursor.execute('SELECT name from sqlite_master where type= "table"')
print(cursor.fetchall())

[('as_house_type',), ('as_info',), ('as_history',), ('as_price_val',), ('as_seller_type',), ('as_seller_info',)]


In [12]:
cursor.execute("PRAGMA table_info('as_history')")
as_info = [name[1] for name in cursor.fetchall()]
print(as_info)

['marker_id', 'price', 'val_id', 'views', 'date']


In [13]:
cursor.execute("PRAGMA table_info('as_info')")
as_info = [name[1] for name in cursor.fetchall()]
print(as_info)

['marker_id', 'href', 'title', 'house_id', 'address', 'seller_id', 'description', 'priority', 'params', 'map_lat', 'map_lon']


In [14]:
from datetime import date
current_date = date.today()
print(current_date)

2022-03-18


In [18]:
current_date = '2022-03-17'

In [19]:
cursor.execute("SELECT href FROM as_info WHERE marker_id IN (SELECT marker_id FROM as_history WHERE date = ?);", (current_date,))
href_list = cursor.fetchall()

In [20]:
href_list = [href[0] for href in href_list]

In [21]:
len(href_list)

325

In [ ]:
#connection.close()

In [22]:
def write_db(data):
    
    house_name = data['kind']
    cursor.execute("SELECT house_id FROM as_house_type WHERE house_name = ?;", (house_name,))
    
    try:
        house_id = cursor.fetchall()[0][0]
    except:
        pass
    
    try:
        cursor.execute("SELECT params, map_lat, map_lon FROM as_info WHERE marker_id = ?;", (href.split('_')[-1],))
        as_info = cursor.fetchall()
        len(as_info[0][0])
    except:
        cursor.execute("UPDATE as_info SET house_id = ?, params = ?, map_lat = ?, map_lon = ? WHERE marker_id = ?;", \
                       (house_id, data['params'], data['map_lat'], data['map_lon'], href.split('_')[-1]))
        connection.commit()
        
    try:
        cursor.execute("UPDATE as_history SET views = ? WHERE marker_id = ? AND date = ?;", (data['views'], href.split('_')[-1], current_date))
        connection.commit()
    except:
        pass

In [23]:
def view_sum(view):
    view = view.lstrip().split()
    if len(view) == 2:
        return int(view[0]) + int(view[1][2:-1])
    else:
        return int(view[0]) 

In [24]:
def get_kvartira_data(main_page):
    kvartira = BeautifulSoup(main_page, 'html.parser')
    try:
        kind = kvartira.find_all('a', {'class':'js-breadcrumbs-link js-breadcrumbs-link-interaction'})[-2].text.replace('\n', '')
    except:
        kind = None
    try:
        if len(kvartira.find_all('div', {'class':'item-view-block'})) == 4:
            params = kvartira.find_all('div', {'class':'item-view-block'})[0].find_all('li', {'class':'item-params-list-item'})
            params = [param.text.strip().replace("\n                    ",'').replace("\xa0",' ') for param in params]
            params_room = ', '.join(params)
        else:
            params_room = kvartira.find_all('div', {'class':'item-view-block'})[1].find_all('li', {'class':'item-params-list-item'})
            params_room = [param.text.strip().replace("\n                    ",'').replace("\xa0",' ') for param in params_room]
            params_room = ', '.join(params_room)
    except:
        params_room = None
    try:
        map_lat = kvartira.find('div', {'class':'b-search-map expanded item-map-wrapper js-item-map-wrapper'})['data-map-lat']
    except:
        map_lat = None
    try:
        map_lon = kvartira.find('div', {'class':'b-search-map expanded item-map-wrapper js-item-map-wrapper'})['data-map-lon'] 
    except:
        map_lon = None
    try:    
        params_house = kvartira.find_all('div', {'class':'item-view-block'})[-1].find_all('li', {'class':'item-params-list-item'})
        params_house = [param.text.strip().replace("\n                    ",'').replace("\xa0",' ') for param in params_house]
        params_house = ', '.join(params_house)
    except:             
        params_house = None
    try:     
        views = kvartira.find('div', {'class':'title-info-metadata-item title-info-metadata-views'}).text
        views = view_sum(views)
    except:    
        views = None 

    params =  params_room + ', ' + params_house
    
    data = {'kind': kind,
            'params' : params,
            'map_lat' : map_lat,
            'map_lon' : map_lon,
            'views' : views}
       
    write_db(data)
    
    return data

In [26]:
for href in href_list: 
    driver.get(href)
    time.sleep(6)
    main_page = driver.page_source
    get_kvartira_data(main_page)

In [27]:
connection.close()